In [1]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

from sklearn.preprocessing import LabelEncoder

### Datasets

In [2]:
apartamentos_2out22 = pd.read_excel('imovirtual_02OUT2022(apartamentos).xlsx')
apartamentos_2out22 = apartamentos_2out22.set_index('ID')
moradias_2out22 = pd.read_excel('imovirtual_02OUT2022(moradias).xlsx')
moradias_2out22 = moradias_2out22.set_index('ID')
quintas_herdades_2out22 = pd.read_excel('imovirtual_02OUT2022(quintas e herdades).xlsx')
quintas_herdades_2out22 = quintas_herdades_2out22.set_index('ID')

moradias_5nov22 = pd.read_excel('imovirtual_05NOV2022(moradias).xlsx')
moradias_5nov22 = moradias_5nov22.set_index('ID')
quintas_herdades_5nov22 = pd.read_excel('imovirtual_05NOV2022(Quintas e Herdades).xlsx')
quintas_herdades_5nov22 = quintas_herdades_5nov22.set_index('ID')

moradias_18mar23 = pd.read_csv('imovirtual_18MAR2023(moradias).csv', delimiter=';')
moradias_18mar23 = moradias_18mar23.set_index('Unnamed: 0')
apartamentos_18mar23 = pd.read_csv('imovirtual_18MAR2023(apartamentos).csv', delimiter=';')
apartamentos_18mar23 = apartamentos_18mar23.set_index('Unnamed: 0')
quintas_herdades_18mar23 = pd.read_csv('imovirtual_18MAR2023(quintas e herdades).csv', delimiter=';')
quintas_herdades_18mar23 = quintas_herdades_18mar23.set_index('Unnamed: 0')
terrenos_18mar23 = pd.read_csv('imovirtual_18MAR2023(terrenos).csv', delimiter=';')
terrenos_18mar23 = terrenos_18mar23.set_index('Unnamed: 0')

In [3]:
apartamentos_18mar23.columns

Index(['Tipo', 'Titulo', 'Preço', 'Condição', 'Quartos', 'WC', 'Dimensão m2',
       'Terreno m2', 'Localidade', 'Link', 'Data'],
      dtype='object')

In [4]:
quintas_herdades_18mar23.columns

Index(['Tipo', 'Titulo', 'Preço', 'Condição', 'Quartos', 'WC', 'Dimensão m2',
       'Terreno m2', 'Localidade', 'Link', 'Data'],
      dtype='object')

#### Merge all except terrenos (has diferente structure)

In [5]:
todos = pd.concat([apartamentos_2out22,moradias_2out22,quintas_herdades_2out22, moradias_5nov22,quintas_herdades_5nov22, moradias_18mar23, apartamentos_18mar23,quintas_herdades_18mar23], axis=0) 

### Treatment

#### Remove 'm²' and '€/m²' from Dimensão m2

In [6]:
# Assume the original dataframe is called 'todos'
# Create a copy of the dataframe to avoid modifying the original data
#todos_copy = todos.copy()

# Define a function to clean the 'Dimensão m2' column
def clean_dimensoes(row):
    value = row['Dimensão m2']
    if type(value) is int or type(value) is float:
        return value
        # Check if the value has 'm²' or '€/m²'
    elif '€/m²' in value:
        # Get the value of the corresponding row in the 'Terreno m2' column
        terreno_m2_value = row['Terreno m2']
        # Remove 'm²' and any spaces from the 
        if (type(terreno_m2_value) is not int and type(terreno_m2_value) is not float) and ('m²' in terreno_m2_value):
            terreno_m2_value = terreno_m2_value.replace('m²', '')#.strip()
        # Return the cleaned 'Terreno m2' value
        return terreno_m2_value
    elif 'm²' in value:
        # Remove 'm²' and any spaces from the value
        return value.replace('m²', '')#.strip()
    
# Apply the 'clean_dimensoes' function to the 'Dimensão m2' column
todos['Dimensão m2'] = todos.apply(clean_dimensoes, axis=1)



#### Remove T from quartos

In [7]:
todos['Quartos'] = todos['Quartos'].str.replace('T', '')

#### Remove spaces from Preço

In [8]:
todos['Preço'] = todos['Preço'].str.replace(' ', '')

#### Drop of features

In [10]:
todos = todos.drop('Terreno m2', axis=1)

#### Treatment of strange values in Condição and WC

In [11]:
todos['Condição'] = todos['Condição'].replace('[]', pd.NA)
todos['WC'] = todos['WC'].replace('[]', pd.NA)

#### Change types

In [12]:

todos['Preço'] = pd.to_numeric(todos['Preço'], errors='coerce').fillna(-99).astype(int)
todos['Quartos'] = pd.to_numeric(todos['Quartos'], errors='coerce').fillna(-99).astype(int)
todos['WC'] = pd.to_numeric(todos['WC'], errors='coerce').fillna(-99).astype(int)
todos['Dimensão m2'] = pd.to_numeric(todos['Dimensão m2'], errors='coerce').fillna(-99).astype(float)

#### Variável Tipos

In [13]:
# create new column with standardized values
todos['Tipo'] = todos['Tipo'].str.replace(' para comprar:', '').str.strip()

In [14]:
le = LabelEncoder()
todos['Tipo_id'] = le.fit_transform(todos['Tipo'])

#### Remove -99 nulls

In [15]:
#todos.replace(-99,pd.NA)
todos['Preço'].replace(-99, np.nan, inplace=True)
todos['Quartos'].replace(-99, np.nan, inplace=True)
todos['WC'].replace(-99, np.nan, inplace=True)
todos['Dimensão m2'].replace(-99, np.nan, inplace=True)

#### Remove properties with negative price and value 1

In [16]:
todos[todos['Preço']<=1]

,Tipo,Titulo,Preço,Condição,Quartos,WC,Dimensão m2,Link,Data,Cidade,Tipo_id
23287,Moradia,Fantástica moradia nova pronta a habitar,1.000000e+00,Novo,4.0,3.0,190.0,https://www.imovirtual.com/pt/anuncio/fantasti...,2022-10-02,Leiria,1
43697,Moradia,Moradia de Luxo na encosta do Bom Jesus pronta...,1.000000e+00,Renovado,4.0,NaN,670.0,https://www.imovirtual.com/pt/anuncio/moradia-...,2022-10-02,Braga,1
165,Moradia,"Moradia Isolada T3 Venda em Joane,Vila Nova de...",1.000000e+00,Em construção,3.0,3.0,380.0,https://www.imovirtual.com/pt/anuncio/moradia-...,2022-11-06,Braga,1
19948,Moradia,Fantástica moradia T3+1 com piscina!,1.000000e+00,Usado,3.0,3.0,236.0,https://www.imovirtual.com/pt/anuncio/fantasti...,2023-03-18,Braga,1
70697,Apartamento,Apartamento T3 Multiusos,1.000000e+00,<NA>,3.0,NaN,154.0,https://www.imovirtual.com/pt/anuncio/apartame...,2023-03-18,Braga,0
79846,Apartamento,Apartamento T2+1 frente ao mar em Buarcos,-2.147484e+09,Renovado,2.0,1.0,222.0,https://www.imovirtual.com/pt/anuncio/apartame...,2023-03-18,Coimbra,0


In [17]:
todos = todos[todos['Preço'] > 1]

In [18]:
todos[todos['Preço']<=1]

,Tipo,Titulo,Preço,Condição,Quartos,WC,Dimensão m2,Link,Data,Cidade,Tipo_id


####  Treat Cidades

In [19]:
todos['Cidade'].unique()

array(['Lisboa', 'Porto', 'Santarém', 'Faro', 'Coimbra', 'Setúbal',
       'Aveiro', 'Madeira', 'Braga', 'Castelo', 'Branco', 'Leiria',
       'Pico', 'Real', 'Évora', 'Santo', 'Miguel', 'Portalegre', 'Guarda',
       'Terceira', 'Viseu', 'Bragança', 'Flores', 'Beja', 'Jorge',
       'Maria', 'Faial', 'Graciosa', 'Corvo'], dtype=object)

In [20]:
todos['Cidade']=todos['Cidade'].replace('Castelo','Viana do Castelo')
todos['Cidade']=todos['Cidade'].replace('Branco','Castelo Branco')
todos['Cidade']=todos['Cidade'].replace('Jorge','São Jorge')
todos['Cidade']=todos['Cidade'].replace('Real','Vila Real')

In [21]:
todos['Cidade'].unique()

array(['Lisboa', 'Porto', 'Santarém', 'Faro', 'Coimbra', 'Setúbal',
       'Aveiro', 'Madeira', 'Braga', 'Viana do Castelo', 'Castelo Branco',
       'Leiria', 'Pico', 'Vila Real', 'Évora', 'Santo', 'Miguel',
       'Portalegre', 'Guarda', 'Terceira', 'Viseu', 'Bragança', 'Flores',
       'Beja', 'São Jorge', 'Maria', 'Faial', 'Graciosa', 'Corvo'],
      dtype=object)

### Freguesias, Concelhos e Distritos

In [ ]:
# Define a function to split the Localidade column
def split_localidade(localidade):
    if localidade.count(',') == 2:
        freguesia = localidade.split(',')[0].strip()
        concelho = localidade.split(',')[1].strip()
        distrito = localidade.split(',')[2].strip()

    elif localidade.count(',') == 1:
        freguesia = localidade.split(',')[0].strip()
        concelho = localidade.split(',')[0].strip()
        distrito = localidade.split(',')[1].strip()

    elif localidade.count(',') == 0:
        freguesia = localidade.strip()
        concelho = localidade.strip()
        distrito = localidade.strip()

    elif localidade.count(',') >= 3:
      freguesia = localidade.rsplit(',', 2)[0].strip()
      concelho = localidade.rsplit(',', 2)[1].strip()
      distrito = localidade.rsplit(',', 2)[2].strip()

    return pd.Series([freguesia, concelho, distrito], index=['Freguesia', 'Concelho', 'Distrito'])

In [ ]:
# Apply the function to the 'Localidade' column of the dataframe
todos[['Freguesia', 'Concelho', 'Distrito']] = todos['Localidade'].apply(split_localidade)

### Save

In [22]:
todos.to_csv('dataset_final.csv', index=False)

In [ ]:
todos.to_excel('dataset_final.xlsx', index=False)